In [98]:
####################
#the code is to replicate the paper "Annual report readability, current earnings,and earnings persistence" by Feng Li, 2008
####################

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 6. Finally, firm-years that have operating earnings (scaled by book value of assets) greater than 1 or less than -1 are deleted from the sample. 
# 7. control variables calculation

In [ ]:
####################
#6. Finally, firm-years that have operating earnings (scaled by book value of assets) greater than 1 or less than -1 are deleted from the sample. 
####################

In [ ]:
import wrds
conn = wrds.Connection()

In [ ]:
comp_funda = conn.raw_sql("""SELECT * 
                          FROM comp.funda
                          WHERE datadate BETWEEN '2014-01-01' AND '2023-12-31'""")
comp_funda.to_csv("comp_funda.csv", index=False)
#directly download the data from wrds would be much faster

In [40]:
firm_year = pd.read_csv('merge.csv' )
firm_year['filing_date'] = pd.to_datetime(firm_year['filing_date'], errors='coerce').dt.strftime('%Y-%m-%d')
firm_annual = pd.read_csv("comp_funda.csv")

#left merge firm_year with firm_annual by datadate, gvkey, cik
merge = pd.merge(firm_year, firm_annual, on=['datadate', 'gvkey', 'cik'], how='left')

/var/folders/4t/w05mrsdn0r9d85szm0g3kg740000gn/T/ipykernel_614/80131807.py:3: DtypeWarning: Columns (26,30,948,949,950,970,975) have mixed types. Specify dtype option on import or set low_memory=False.
  firm_annual = pd.read_csv("comp_funda.csv")


In [41]:
#Earnings is operating earnings (data178 of Compustat: oiadp) scaled by book value of assets. at
#Finally, firm-years that have operating earnings (scaled by book value of assets) greater than 1 or less than - 1 are deleted from the sample.
#oiadp/at
merge['Earnings'] = merge['oiadp'] / merge['at']
merge = merge[(merge['Earnings'] <= 1) & (merge['Earnings'] >= -1)]

In [42]:
#data6 : at
#data25 : csho
#data181 : lt
#data199 : prcc
#data178: oiadp

In [43]:
#Market-to-book is the market value of the firm divided by its book value ((data25 * data199 + data181)/data6).   (market value of equity + total debt) / total assets. 
#Market value of equity is calculated as (data25 * data199). 
#Size is the logarithm of market value of equity calculated as Log(data25 * data199). 
#Book value of assets is data6 from Compustat. at
#mtb = ((prcc_f * csho - ceq + at)/at)
#mve = prcc_f * csho = data 199(Price – Fiscal Year – Close) * data 25(Common Shares Outstanding)
#size = np.log(prcc_f * csho)

merge['mtb'] = (merge['prcc_f'] * merge['csho'] - merge['lt'])/ merge['at']
merge['mve'] = merge['prcc_f'] * merge['csho']
merge['size'] = np.log(merge['prcc_f'] * merge['csho'])

merge.to_csv("merge_v.csv", index=False)

In [ ]:
#Financial complexity
# num of BUSSEG/GEOSEG
#count the sid by gvkey and datadate
#NBSEG = log(number of business segments + 1)
#NGSEG = log(number of geographic segments + 1)
segments = pd.read_csv("segments.csv")
nbseg = segments[segments['stype'] == 'BUSSEG'].groupby(['cik', 'datadate', 'gvkey'])['sid'].count().reset_index(name='NBSEG')
nbseg['NBSEG'] = np.log(nbseg['NBSEG'] + 1)
ngseg = segments[segments['stype'] == 'GEOSEG'].groupby(['cik', 'datadate', 'gvkey'])['sid'].count().reset_index(name='NGSEG')
ngseg['NGSEG'] = np.log(ngseg['NGSEG'] + 1)

segments = segments.merge(nbseg, on=['cik', 'datadate', 'gvkey'], how='left')
segments = segments.merge(ngseg, on=['cik', 'datadate', 'gvkey'], how='left')


segments['datadate'] = pd.to_datetime(segments['datadate'], errors='coerce').dt.strftime('%Y-%m-%d')
segments = segments[['cik','datadate','gvkey','NBSEG','NGSEG']]
segments = segments.drop_duplicates(subset=['cik', 'datadate', 'gvkey', 'NBSEG', 'NGSEG'])

/var/folders/4t/w05mrsdn0r9d85szm0g3kg740000gn/T/ipykernel_27241/3841153633.py:6: DtypeWarning: Columns (34,45) have mixed types. Specify dtype option on import or set low_memory=False.
  segments = pd.read_csv("segments.csv")


In [46]:
merge = pd.read_csv("merge_v.csv")
merge['datadate'] = pd.to_datetime(merge['datadate'], errors='coerce').dt.strftime('%Y-%m-%d')
merge = pd.merge(merge, segments, on=['cik','datadate','gvkey'], how='left')
merge = merge.reset_index(drop=True)
merge.to_csv("merge_gh.csv", index=False)

/var/folders/4t/w05mrsdn0r9d85szm0g3kg740000gn/T/ipykernel_614/254682104.py:1: DtypeWarning: Columns (44,961,983) have mixed types. Specify dtype option on import or set low_memory=False.
  merge = pd.read_csv("merge_v.csv")


In [11]:
####################
# 7. control variables
####################

In [12]:
merge = pd.read_csv("merge_gh.csv") 
merge['datayear'] = pd.to_datetime(merge['datadate'], errors='coerce').dt.strftime('%Y')

C:\Users\chenw\AppData\Local\Temp\ipykernel_20164\1024775092.py:1: DtypeWarning: Columns (44,961,983) have mixed types. Specify dtype option on import or set low_memory=False.
  merge = pd.read_csv("merge_gh.csv")


In [13]:
#Firm age:
year = pd.read_csv("year.csv")
year['year'] = pd.to_datetime(year['date'], errors='coerce').dt.strftime('%Y')
year = year.groupby('PERMNO')['year'].min().reset_index()
year.rename(columns={'year': 'early_year'}, inplace=True)
year.head

merge = pd.merge(merge, year, left_on=['permno'], right_on=['PERMNO'], how='left')
merge['age'] = pd.to_numeric(merge['datayear'], errors='coerce') - pd.to_numeric(merge['early_year'], errors='coerce')

In [14]:
#Special items: si = spi(data17)/at
#DLW is a dummy that equals 1 if a company is incorporated in Delaware and 0 otherwise.
merge['si'] = merge['spi']/merge['at']
merge['DLW'] = (merge['incorp'] == 'DE').astype(int)

In [15]:
#RET_VOL standard deviation of the monthly stock returns in the prior year
mon_ret = pd.read_csv("mon_ret.csv")
mon_ret['date'] = pd.to_datetime(mon_ret['date'], errors='coerce').dt.strftime('%Y-%m')
mon_ret = mon_ret.sort_values(by=['PERMNO', 'date'])
mon_ret['RET'] = pd.to_numeric(mon_ret['RET'], errors='coerce')
mon_ret['RET_VOL'] = mon_ret.groupby('PERMNO')['RET'].rolling(window=12, min_periods=12).std().reset_index(level=0, drop=True)

merge['ym'] = pd.to_datetime(merge['datadate'], errors='coerce').dt.strftime('%Y-%m')
merge = pd.merge(merge, mon_ret, left_on=['permno','ym'], right_on=['PERMNO','date'], how='left')

In [ ]:
#Volatility of business or operations: EARN_VOL standard deviation of the operating earnings(oiadp 178) in the last five fiscal years
y_EARN = pd.read_csv("oiadp_vol.csv")
y_EARN = y_EARN.sort_values(by=['gvkey', 'fyear'])
y_EARN = (y_EARN.loc[y_EARN.groupby(['gvkey', 'fyear'])['datadate'].idxmax()].reset_index(drop=True))
y_EARN['Earnings'] = y_EARN['oiadp'] / y_EARN['at']
y_EARN['EARN_VOL'] = y_EARN.groupby('gvkey')['Earnings'].rolling(window=5, min_periods=5).std().reset_index(level=0, drop=True)
y_EARN = y_EARN[['gvkey','datadate','EARN_VOL']]

merge['datadate'] = pd.to_datetime(merge['datadate'], errors='coerce').dt.strftime('%Y-%m-%d')
merge = pd.merge(merge, y_EARN, on=['gvkey','datadate'], how='left')

In [ ]:
#NITEMS is the number of non-missing items on Compustat. 
compustat_fs = pd.read_csv("compustat_fs.csv")
compustat_fs['NITEMS'] = compustat_fs[compustat_fs.columns[9:784]].apply(lambda x: x.count(), axis=1)
compustat_fs['datadate'] = pd.to_datetime(compustat_fs['datadate'], errors='coerce').dt.strftime('%Y-%m-%d')
compustat_fs = compustat_fs[['gvkey','datadate','NITEMS']]
compustat_fs = compustat_fs.loc[compustat_fs.groupby(['gvkey', 'datadate'])['NITEMS'].idxmax()].reset_index(drop=True)

merge = pd.merge(merge, compustat_fs, on=['gvkey','datadate'], how='left')

In [18]:
#MA is a dummy that equals 1 if a firm appears as an acquirer in this year in SDC Platinum M&A database and 0 otherwise. 
sdc_ma = pd.read_csv("sdc_ma.csv")
sdc_ma['year'] = pd.to_datetime(sdc_ma['hdate'], errors='coerce').dt.strftime('%Y')
sdc_ma = sdc_ma.drop_duplicates(subset=['year','ACUSIP'])
sdc_ma = sdc_ma[['year','ACUSIP']]

merge['CUSIP_6'] = merge['cusip'].str[:6]
merge = pd.merge(merge, sdc_ma, left_on =['CUSIP_6','datayear'], right_on =['ACUSIP','year'], how='left')
merge['MA'] = (merge['ACUSIP'].notnull()).astype(int)

In [19]:
#SEO is set to 1 for a year in which a company has a common equity offering in the secondary market according to the SDC Global New Issues database
#seconshares:Common Stock Items: Secondary Shares (seconshares)
sdc_new = pd.read_csv("sdc_new.csv")
sdc_new['SEO'] = (sdc_new['SECONSHARES'] > 0).astype(int)
sdc_new['year'] = pd.to_datetime(sdc_new['MASTER_DEAL_DATE'], errors='coerce').dt.strftime('%Y')
sdc_new['CUSIP'] = sdc_new['CUSIP'].astype(str).str[:6]
sdc_new1 = sdc_new[['CUSIP','SEO','year']].drop_duplicates()
sdc_new2 = sdc_new[['CUSIP9','SEO','year']].drop_duplicates()

# Perform the merge operations
merge = pd.merge(merge, sdc_new1, left_on=['CUSIP_6', 'datayear'], right_on=['CUSIP', 'year'], how='left', suffixes=('', '_sdc1'))
merge = pd.merge(merge, sdc_new2, left_on=['CUSIP', 'datayear'], right_on=['CUSIP9', 'year'], how='left', suffixes=('', '_sdc2'))

# Fill missing values in the `SEO` column
merge['SEO'] = merge['SEO'].fillna(0)

C:\Users\chenw\AppData\Local\Temp\ipykernel_20164\1365115732.py:3: DtypeWarning: Columns (23,39,40,53,86) have mixed types. Specify dtype option on import or set low_memory=False.
  sdc_new = pd.read_csv("sdc_new.csv")


In [20]:
merge = merge[['gvkey','datadate','datayear','cik','filing_date','url','fog_index','length','num_words','mda_fog_index',
               'mda_length','mda_words','bogindex','sic','at','Earnings','mve', 'mtb', 'size','age',
               'NBSEG', 'NGSEG','si','RET_VOL', 'EARN_VOL','DLW', 'NITEMS','MA','SEO']]
merge = merge.drop_duplicates()
merge = merge.loc[merge.groupby(['gvkey','filing_date','datadate'])['age'].idxmax()].reset_index(drop=True)

merge.to_csv("final.csv")

In [ ]:
#path:
#segments: Segments (Non-Historical)
#year.csv crsp_a_stock/msf
#mon_ret.csv:crsp_a_stock/msf(Monthly Stock - Securities)
#oiadp_vol:comp_na_daily_all/funda
#compustat_fs:comp/comp_na_daily_all/funda
#sdc_ma: tr_sdc_ma/wrds_ma_events
#sdc_new: tr_sdc_ni/wrds_ni_details